
# Web Scraping with Beautiful Soup: Solutions

In [1]:
# Import required libraries
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

**Explicación de cada import**

**from bs4 import BeautifulSoup**
- Trae la clase BeautifulSoup de la librería BeautifulSoup4.
- Sirve para parsear (analizar) el HTML o XML de páginas web y poder extraer información (textos, tablas, enlaces, etc.).

**from datetime import datetime**
- Importa la clase datetime.
- Te permite manejar fechas y horas: obtener la fecha actual, dar formato, calcular diferencias de tiempo, etc.

**import requests**
- Carga la librería requests, usada para hacer peticiones HTTP (GET, POST, etc.).
- Es lo que se usa para descargar contenido de páginas web o conectarse a APIs.

**import time**
- Importa el módulo estándar time.
- Te permite trabajar con funciones relacionadas al tiempo:

        * time.sleep(segundos) para pausar la ejecución.
        * time.time() para medir tiempos de ejecución.

In [2]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp')
# Read the content of the server’s response
src = req.text
# Parse the response into an HTML tree
soup = BeautifulSoup(src, 'lxml')

**Explicacion**

**req = requests.get('http://www.ilga.gov/senate/default.asp')**

* Usa requests para hacer una petición HTTP GET a la página del Senado del Estado de Illinois (ilga.gov).
* El servidor responde con el código HTML de esa página.
* Ese resultado queda guardado en req.
* Puedes verificar si la petición fue exitosa con req.status_code (200 significa OK).

**src = req.text**
* Extrae el contenido de la respuesta en forma de texto (string).
* Aquí estará el HTML crudo de la página web.

**soup = BeautifulSoup(src, 'lxml')**
* Convierte el HTML en un árbol DOM navegable con la librería BeautifulSoup.
* Usa el parser lxml (más rápido y flexible que html.parser).
* Ahora puedes recorrer el HTML con métodos como .find(), .find_all(), .title, etc.

## Challenge: Find All

Use Beautiful Soup to find all the `a` elements with class `mainmenu`.

In [3]:
soup.select("a.mainmenu")

[]

**Explicacion del codigo**

    Con soup.select("a.mainmenu") obtendrás una lista de elementos <'a'> solo con clase mainmenu, es decir los dos primeros enlaces.

## Challenge: Extract Specific Attributes

Extract all `href` attributes for each `mainmenu` URL.

In [ ]:
[link['href'] for link in soup.select("a.mainmenu")]

**Explicacion del codigo**

    Ese código lista para extraer solo los atributos href (los enlaces) de todas las etiquetas <'a'> con clase mainmenu.

## Challenge: Get `href` elements pointing to members' bills

The code above retrieves information on:  

- the senator's name,
- their district number,
- and their party.

We now want to retrieve the URL for each senator's list of bills. Each URL will follow a specific format. 

The format for the list of bills for a given senator is:

`http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=[MEMBER_ID]&Primary=True`

to get something like:

`http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True`

in which `MEMBER_ID=1911`. 

You should be able to see that, unfortunately, `MEMBER_ID` is not currently something pulled out in our scraping code.

Your initial task is to modify the code above so that we also **retrieve the full URL which points to the corresponding page of primary-sponsored bills**, for each member, and return it along with their name, district, and party.

Tips: 

* To do this, you will want to get the appropriate anchor element (`<a>`) in each legislator's row of the table. You can again use the `.select()` method on the `row` object in the loop to do this — similar to the command that finds all of the `td.detail` cells in the row. Remember that we only want the link to the legislator's bills, not the committees or the legislator's profile page.
* The anchor elements' HTML will look like `<a href="/senate/Senator.asp/...">Bills</a>`. The string in the `href` attribute contains the **relative** link we are after. You can access an attribute of a BeatifulSoup `Tag` object the same way you access a Python dictionary: `anchor['attributeName']`. See the <a href="http://www.crummy.com/software/BeautifulSoup/bs4/doc/#tag">documentation</a> for more details.
* There are a _lot_ of different ways to use BeautifulSoup to get things done. whatever you need to do to pull the `href` out is fine.

The code has been partially filled out for you. Fill it in where it says `#YOUR CODE HERE`. Save the path into an object called `full_path`.

In [5]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")
# Create empty list to store our data
members = []

# Returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')
# Get rid of junk rows
rows = [row for row in rows if row.select('td.detail')]

# Loop through all rows
for row in rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.detail') 
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]
    
    # YOUR CODE HERE
    # Extract href
    href = row.select('a')[1]['href']
    # Create full path
    full_path = "http://www.ilga.gov/senate/" + href + "&Primary=True"
    
    # Store in a tuple
    senator = (name, district, party, full_path)
    # Append to list
    members.append(senator)

Ese bloque de código es un scraper de la página del Senado de Illinois (98ª Asamblea General). Lo que hace es recorrer la tabla de senadores y extraer sus datos principales. Construye una lista de senadores de Illinois (98th GA) con:

* Nombre
* Distrito
* Partido político
* Enlace a su perfil oficial

In [7]:
import re, time
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

LIST_URLS = [
    "https://www.ilga.gov/Senate/Members/List",
    "https://www.ilga.gov/Senate/List",
]
BASE = "https://www.ilga.gov"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "es-ES,es;q=0.9,en;q=0.8",
    "Referer": "https://www.ilga.gov/",
    "Connection": "keep-alive",
}

def fetch(url):
    s = requests.Session()
    s.headers.update(HEADERS)
    r = s.get(url, timeout=30)
    r.raise_for_status()
    return r

def find_profile_links(html, base=BASE):
    soup = BeautifulSoup(html, "lxml")
    # busca anchors a /Senate/Members/Details/... (insensible a mayúsculas)
    links = set()
    for a in soup.select('a[href]'):
        href = a.get("href") or ""
        if re.search(r"/Senate/Members/Details/\d+", href, flags=re.I):
            links.add(urljoin(base, href))
    return sorted(links)

def parse_profile(html):
    """Devuelve (name, district:int|None, party:str|''), usando varias heurísticas."""
    soup = BeautifulSoup(html, "lxml")

    # Nombre: probar h1, h2, title, aria-labels…
    name = ""
    cand = []
    cand += [h.get_text(strip=True) for h in soup.select("h1")]
    if not cand: cand += [h.get_text(strip=True) for h in soup.select("h2")]
    if not cand and soup.title: cand += [soup.title.get_text(strip=True)]
    if cand: name = cand[0]

    # Texto visible para regex
    text = soup.get_text(" ", strip=True)

    # Distrito (varias variantes)
    district = None
    for pat in [
        r"District\s*(\d+)",
        r"(\d+)\s+District",
        r"Senate\s+District\s*(\d+)",
        r"(\d+)\s+\(D\)|(\d+)\s+\(R\)|(\d+)\s+\(I\)"
    ]:
        m = re.search(pat, text, flags=re.I)
        if m:
            # toma el primer grupo no vacío
            for g in m.groups():
                if g and g.isdigit():
                    district = int(g)
                    break
        if district is not None:
            break

    # Partido (D/R/I o completo)
    party = ""
    # primero letra entre paréntesis
    m = re.search(r"\((D|R|I)\)", text)
    if m:
        party = m.group(1)
    else:
        # palabras completas
        if re.search(r"\bDemocrat(ic)?\b", text, flags=re.I):
            party = "D"
        elif re.search(r"\bRepublican\b", text, flags=re.I):
            party = "R"
        elif re.search(r"\bIndependent\b", text, flags=re.I):
            party = "I"

    return name, district, party

# === Flujo principal ===
try:
    # 1) Prueba con las URLs de lista y junta todos los perfiles
    all_profiles = []
    for url in LIST_URLS:
        try:
            r = fetch(url)
            profs = find_profile_links(r.text)
            if profs:
                all_profiles.extend(profs)
        except Exception as e:
            print(f"[WARN] No se pudo leer {url}: {e}")

    # de-duplicar
    all_profiles = sorted(set(all_profiles))
    print("Perfiles encontrados en la lista:", len(all_profiles))

    # Diagnóstico si no hay enlaces
    if not all_profiles:
        print("No se hallaron enlaces a perfiles. Guardando debug_list.html…")
        try:
            open("debug_list.html", "w", encoding="utf-8").write(r.text)
            print("Revisa debug_list.html para ver el HTML real (¿portal cautivo/bloqueo?).")
        except Exception:
            pass

    # 2) Visita cada perfil y extrae datos
    members = []
    for i, purl in enumerate(all_profiles, 1):
        try:
            pr = fetch(purl)
            name, district, party = parse_profile(pr.text)
            if name:  # al menos nombre
                members.append((name, district, party, purl))
        except Exception as e:
            print(f"[WARN] Perfil con error ({purl}): {e}")
        time.sleep(0.5)  # pausa cortita para ser amable

    print("Total miembros parseados:", len(members))

    # 3) Mostrar algunas filas
    for m in members[:10]:
        print(m)

    # 4) (Opcional) DataFrame/CSV
    try:
        import pandas as pd
        df = pd.DataFrame(members, columns=["Nombre", "Distrito", "Partido", "Perfil"])
        print("\nPrimeras 5 filas:")
        print(df.head())
        df.to_csv("senado_ilga_moderno.csv", index=False, encoding="utf-8")
        print("\nCSV generado: senado_ilga_moderno.csv")
    except ImportError:
        print("Pandas no está instalado; omitiendo CSV. Instala con: pip install pandas openpyxl")

except Exception as e:
    print("Error general:", e)

[WARN] No se pudo leer https://www.ilga.gov/Senate/List: 404 Client Error: Not Found for url: https://www.ilga.gov/Senate/List
Perfiles encontrados en la lista: 60
Total miembros parseados: 60
('Member', 8505, 'D', 'https://www.ilga.gov/Senate/Members/Details/3264')
('Member', 5413, 'R', 'https://www.ilga.gov/Senate/Members/Details/3265')
('Member', 8176, 'D', 'https://www.ilga.gov/Senate/Members/Details/3268')
('Member', 5966, 'D', 'https://www.ilga.gov/Senate/Members/Details/3269')
('Member', 422, 'D', 'https://www.ilga.gov/Senate/Members/Details/3270')
('Member', 8250, 'D', 'https://www.ilga.gov/Senate/Members/Details/3271')
('Member', 9573, 'D', 'https://www.ilga.gov/Senate/Members/Details/3276')
('Member', 3840, 'R', 'https://www.ilga.gov/Senate/Members/Details/3281')
('Member', 5145, 'D', 'https://www.ilga.gov/Senate/Members/Details/3291')
('Member', 8066, 'D', 'https://www.ilga.gov/Senate/Members/Details/3292')

Primeras 5 filas:
   Nombre  Distrito Partido                      

**Explicacion del codigo**

Este script es un scraper a la pagina del Senado de Illinois (ILGA). Hace esto:

**Configura**
* LIST_URLS: páginas de listado donde buscar enlaces a perfiles.
* HEADERS: cabeceras “de navegador” para evitar bloqueos simples.

**Funciones clave**
* fetch(url): hace GET con requests.Session + cabeceras y lanza error si no es 200.
* find_profile_links(html): en el HTML del listado, busca todos los enlaces que coinciden con /Senate/Members/Details <'id'> y devuelve las URLs absolutas (evita dependencias del HTML antiguo).
* parse_profile(html): abre cada perfil individual y, con heurísticas (regex):
    * extrae name (buscando h1, h2 o <'title'>),
    * detecta district (varios patrones: “District 47”, “47 District”, etc.),
    * detecta party (letra entre paréntesis (D|R|I) o palabras “Democrat/Republican/Independent”).

**Flujo principal**
* Visita cada URL en LIST_URLS, junta y de-duplica todas las URLs de perfiles.
* Si no encuentra enlaces, guarda debug_list.html para inspeccionar el HTML real (útil si hay portal cautivo/antibot).
* Recorre cada perfil y arma members con tuplas (Nombre, Distrito, Partido, Perfil), poniendo una pausa de 0.5s por cortesía.
* (Opcional) Crea un DataFrame y exporta senado_ilga_moderno.csv.

**Salida**
* Imprime cuántos **perfiles encontró** y **miembros parseados**, muestra los primeros, y guarda el CSV si pandas está instalado.

## Challenge: Modularize Your Code

Turn the code above into a function that accepts a URL, scrapes the URL for its senators, and returns a list of tuples containing information about each senator. 

In [17]:
def get_members(url):
    # Make a GET request
    req = requests.get(url)
    # Read the content of the server’s response
    src = req.text
    # Soup it
    soup = BeautifulSoup(src, "lxml")
    # Create empty list to store our data
    members = []

    # Returns every ‘tr tr tr’ css selector in the page
    rows = soup.select('tr tr tr')
    # Get rid of junk rows
    rows = [row for row in rows if row.select('td.detail')]

    # Loop through all rows
    for row in rows:
        # Select only those 'td' tags with class 'detail'
        detail_cells = row.select('td.detail') 
        # Keep only the text in each of those cells
        row_data = [cell.text for cell in detail_cells]
        # Collect information
        name = row_data[0]
        district = int(row_data[3])
        party = row_data[4]

        # YOUR CODE HERE
        # Extract href
        href = row.select('a')[1]['href']
        # Create full path
        full_path = "https://www.ilga.gov/Senate/Members/List/" + href + "&Primary=True"

        # Store in a tuple
        senator = (name, district, party, full_path)
        # Append to list
        members.append(senator)
    return(members)

**Explicacion del Codigo**

Ese código define una función get_members(url) que hace un web scraping sobre una página de la Asamblea Legislativa de Illinois. Hace lo siguiente:

**Descarga la página web**
* Hace una petición HTTP a la URL.
* Obtiene el HTML como texto.
* Usa BeautifulSoup con el parser lxml para convertirlo en un árbol de etiquetas.

**Inicializa una lista vacía**
* Aquí se guardarán los senadores extraídos.

**Busca filas de tabla**
* Selecciona todas las filas <'tr'> anidadas tres niveles (muy específico).
* Filtra solo las que contienen celdas <'td class="detail">, que son las filas con información útil.

**Extrae los datos de cada fila**
* De cada fila toma solo las celdas con clase detail.
* Convierte su contenido a texto.
* Interpreta la primera celda como nombre, la cuarta como distrito (convertido a entero), y la quinta como partido político.

**Obtiene el enlace al perfil**
* Busca los enlaces <'a> dentro de la fila.
* Toma el segundo enlace ([1]).
* Construye una URL completa concatenando con una ruta base y agregando &Primary=True.

**Guarda la información en una tupla**
* Agrupa los datos en una tupla con la forma: (Nombre, Distrito, Partido, URL_perfil)
* Lo agrega a la lista members.

**Devuelve el resultado**
* Retorna la lista completa de senadores encontrados en la página.

In [37]:
import re
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

def get_members(url: str = "https://www.ilga.gov/Senate/Members/List"):
    """
    Devuelve una lista de tuplas (Nombre, Distrito:int, Partido:str, Perfil:str)
    extraídas desde la página moderna del Senado de Illinois.
    """
    headers = {"User-Agent": "Mozilla/5.0"}
    resp = requests.get(url, headers=headers, timeout=30)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "lxml")
    members = []

    # Enlaces a perfiles /Senate/Members/Details/<id>
    for a in soup.select('a[href*="/Senate/Members/Details/"], a[href*="/senate/members/details/"]'):
        name = a.get_text(strip=True)
        if not name:
            continue

        # En el contenedor suele venir "Nombre 47 R" (número + partido)
        full_text = a.parent.get_text(" ", strip=True)
        tail = full_text.replace(name, "").strip()

        # 1) patrón directo: "47 R"
        m = re.search(r'(\d+)\s+([DRI])\b', tail)
        # 2) fallback: "District 47 (R)" u otras variantes
        if not m:
            m = re.search(r'(?:District\s*)?(\d+).*?([DRI])\b', tail, re.I)
        if not m:
            # si no se detecta distrito/partido, igual guarda el nombre y el perfil
            district, party = None, ""
        else:
            district = int(m.group(1))
            party = m.group(2).upper()

        profile = urljoin(url, a.get("href"))
        members.append((name, district, party, profile))

    return members


**Explicación del Codigo**

Ese código define una función get_members que hace web scraping sobre la página moderna del Senado de Illinois y devuelve información básica de cada senador. Hace lo siguiente:

**Definicios de Imports**
* re: para usar expresiones regulares.
* urljoin: para construir URLs absolutas a partir de relativas.
* requests: para hacer la descarga de la página web.
* BeautifulSoup: para parsear el HTML.

**Definición de la función**
* Toma como parámetro opcional la URL del listado de miembros del Senado.
* Si no le pasas nada, usa la lista oficial de senadores.

**Descarga y parseo**
* Descarga la página con un User-Agent “realista”.
* raise_for_status() lanza error si la respuesta no fue 200 OK.
* Convierte el HTML a un árbol con BeautifulSoup.

**Inicializa lista de resultados**
**Encuentra enlaces a perfiles individuales**
* Busca todos los <'a> cuyo href contenga /Senate/Members/Details/.
* Extrae el texto (nombre del senador).

**Extrae distrito y partido del texto alrededor**
* El contenedor donde está el link suele tener “Nombre 47 R”.
* Usa regex para detectar el número de distrito y la letra del partido:
    * D = Demócrata
    * R = Republicano
    * I = Independiente
* Si no encuentra nada, pone district = None y party = "".

**Construye la URL del perfil**
* profile = urljoin(url, a.get("href"))

**Guarda la información**
* members.append((name, district, party, profile))

**Devuelve el resultado**
* Entrega una lista de tuplas de la forma: (Nombre, Distrito, Partido, URL_perfil)

In [38]:
senate_members = get_members()  # o get_members("https://www.ilga.gov/Senate/Members/List")
print("Total miembros:", len(senate_members))
for m in senate_members[:5]:
    print(m)


Total miembros: 60
('Neil Anderson', None, '', 'https://www.ilga.gov/Senate/Members/Details/3312')
('Omar Aquino', None, '', 'https://www.ilga.gov/Senate/Members/Details/3316')
('Li Arellano, Jr.', None, '', 'https://www.ilga.gov/Senate/Members/Details/3383')
('Chris Balkema', None, '', 'https://www.ilga.gov/Senate/Members/Details/3413')
('Christopher Belt', None, '', 'https://www.ilga.gov/Senate/Members/Details/3337')



**Explicacion del Codigo**

Este código ejecuta tu scraper, imprime el número total de senadores encontrados, y luego muestra en consola los primeros 5 registros (nombre, distrito, partido y link al perfil).

In [39]:
import pandas as pd

df = pd.DataFrame(senate_members, columns=["Nombre", "Distrito", "Partido", "Perfil"])
print(df.head())
# df.to_csv("senado_ilga_moderno.csv", index=False, encoding="utf-8")


             Nombre Distrito Partido  \
0     Neil Anderson     None           
1       Omar Aquino     None           
2  Li Arellano, Jr.     None           
3     Chris Balkema     None           
4  Christopher Belt     None           

                                             Perfil  
0  https://www.ilga.gov/Senate/Members/Details/3312  
1  https://www.ilga.gov/Senate/Members/Details/3316  
2  https://www.ilga.gov/Senate/Members/Details/3383  
3  https://www.ilga.gov/Senate/Members/Details/3413  
4  https://www.ilga.gov/Senate/Members/Details/3337  


**Explicacion del Codigo**

* Este código convierte tu lista de senadores en una tabla con pandas y te muestra las primeras 5 filas.

## Take-home Challenge: Writing a Scraper Function

We want to scrape the webpages corresponding to bills sponsored by each bills.

Write a function called `get_bills(url)` to parse a given bills URL. This will involve:

  - requesting the URL using the <a href="http://docs.python-requests.org/en/latest/">`requests`</a> library
  - using the features of the `BeautifulSoup` library to find all of the `<td>` elements with the class `billlist`
  - return a _list_ of tuples, each with:
      - description (2nd column)
      - chamber (S or H) (3rd column)
      - the last action (4th column)
      - the last action date (5th column)
      
This function has been partially completed. Fill in the rest.

In [55]:
import re, time
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

HEADERS = {"User-Agent": "Mozilla/5.0"}

def get_members():
    url = "https://www.ilga.gov/Senate/Members/List"
    s = requests.Session(); s.headers.update(HEADERS)
    r = s.get(url, timeout=30); r.raise_for_status()
    soup = BeautifulSoup(r.text, "lxml")
    
    

    # Encuentra todos los enlaces a perfiles
    profile_links = []
    for a in soup.select('a[href]'):
        href = a.get("href") or ""
        if re.search(r"/Senate/Members/Details/\d+", href, flags=re.I):
            profile_links.append(urljoin(url, href))
    profile_links = sorted(set(profile_links))

    members = []
    # Si hay enlaces, visita cada perfil y extrae datos básicos
    for purl in profile_links:
        try:
            pr = s.get(purl, timeout=30); pr.raise_for_status()
            psoup = BeautifulSoup(pr.text, "lxml")
            # Nombre
            name = ""
            if (h1 := psoup.select_one("h1")): name = h1.get_text(strip=True)
            elif (h2 := psoup.select_one("h2")): name = h2.get_text(strip=True)
            elif psoup.title: name = psoup.title.get_text(strip=True)

            text = psoup.get_text(" ", strip=True)
            # Distrito
            m = re.search(r"District\s*(\d+)", text, re.I) or re.search(r"Senate\s+District\s*(\d+)", text, re.I)
            district = int(m.group(1)) if m else None
            # Partido
            party = ""
            m = re.search(r"\((D|R|I)\)", text)
            if m: party = m.group(1)
            elif re.search(r"\bDemocrat(ic)?\b", text, re.I): party = "D"
            elif re.search(r"\bRepublican\b", text, re.I): party = "R"
            elif re.search(r"\bIndependent\b", text, re.I): party = "I"

            if name:
                members.append((name, district, party, purl))
        except Exception:
            continue
        time.sleep(0.2)  # ser amable con el servidor
    return members

**Explicacion del Codigo**

* La función get_members() descarga el listado de senadores de Illinois, detecta los enlaces a cada perfil, entra en cada uno, y devuelve una lista con nombre, distrito, partido y URL de perfil de todos los miembros.

In [56]:
senate_members = get_members()
if not senate_members:
    raise RuntimeError("No se obtuvieron senadores. Revisa debug_list.html y los selectores.")

# Ahora es seguro acceder al primero
test_url = senate_members[0][3]
print("Perfil seleccionado:", test_url)

# Solo llama get_bills si tienes una URL válida
bills = get_bills(test_url)
print(bills[:5])


Perfil seleccionado: https://www.ilga.gov/Senate/Members/Details/3264
[]


**Explicacion del Codigo**

Ese fragmento de código es un bloque de prueba que usa tu función get_members() (la que scrapea la web del Senado) y después llama a otra función get_bills() (seguramente hecha por ti para extraer proyectos de ley de un senador).

Hace lo siguiente:
* Usa get_members() para obtener senadores.
* Se asegura de que haya resultados.
* Selecciona el perfil del primer senador.
* Llama a get_bills() sobre ese perfil.
* Imprime los primeros 5 proyectos de ley de ese senador.

### Scrape All Bills

Finally, create a dictionary `bills_dict` which maps a district number (the key) onto a list of bills (the value) coming from that district. You can do this by looping over all of the senate members in `members_dict` and calling `get_bills()` for each of their associated bill URLs.

**NOTE:** please call the function `time.sleep(1)` for each iteration of the loop, so that we don't destroy the state's web site.

In [60]:
bills_dict = {}  
for member in senate_members[:5]:
    bills_dict[member[1]] = get_bills(member[3])
    time.sleep(1)

In [61]:
# Verificar claves
for key in bills_dict.keys():
    print("Clave encontrada:", key)

# Acceder de forma segura
if 52 in bills_dict:
    print("Número de proyectos para distrito 52:", len(bills_dict[52]))
else:
    print("No hay datos para distrito 52.")


Clave encontrada: None
No hay datos para distrito 52.


In [63]:
print("Claves disponibles en bills_dict:", list(bills_dict.keys()))
print("Total claves:", len(bills_dict))

bills_52 = bills_dict.get(52) or bills_dict.get("52")
if bills_52 is None:
    print("No hay datos para el distrito 52.")
else:
    print("Número de proyectos:", len(bills_52))


Claves disponibles en bills_dict: [None]
Total claves: 1
No hay datos para el distrito 52.
